# My Chromagram

Process raw audio `.wav` files with matching meter maps from `data/derived/meter`, compute metric-aligned chromagrams, visualize results, and save to `data/derived/chromagram`.

In [2]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import librosa

project_root = Path.cwd()
if not (project_root / "src" / "dijon").exists():
    project_root = project_root.parent
sys.path.insert(0, str(project_root))

from dijon.chromagram import metric_chromagram_mvp
from dijon.global_config import DERIVED_DIR, RAW_AUDIO_DIR

%matplotlib inline

In [4]:
HOP_LENGTH = 256
BPM_THRESHOLD = 180.0
CHROMA_TYPE = "cqt"           # "cqt" or "stft"
AGGREGATE = "mean"            # "mean" or "median"
ACCENT_MODE = "preserve"      # "preserve", "normalize", "weighted"
WEIGHT_SOURCE = "rms"         # "rms" or "onset" (weighted mode only)
WEIGHT_POWER = 1.0
MIN_FRAMES_PER_BIN = 2

METER_DIR = DERIVED_DIR / "meter"
CHROMA_OUTPUT_DIR = DERIVED_DIR / "chromagram"

OVERRIDE_FILES = []  # set to [Path(".../YTB-005.wav")] to process specific files


def _track_name(audio_path: Path) -> str:
    return audio_path.stem


if OVERRIDE_FILES:
    audio_paths = [Path(p).resolve() for p in OVERRIDE_FILES]
else:
    audio_paths = sorted(RAW_AUDIO_DIR.glob("*.wav"))

pairs = []
for audio_path in audio_paths:
    track_name = _track_name(audio_path)
    meter_path = METER_DIR / f"{track_name}_meter.npy"
    if meter_path.exists():
        pairs.append((track_name, audio_path, meter_path))

print(f"Found {len(pairs)} track(s) with matching meter maps")
for track_name, audio_path, meter_path in pairs:
    print(f"  {track_name}: audio={audio_path.name} meter={meter_path.name}")

chroma_results = {}
for track_name, audio_path, meter_path in pairs:
    y, sr = librosa.load(audio_path, sr=None, mono=True)
    meter_map = np.load(meter_path).astype(np.float64)

    C_metric = metric_chromagram_mvp(
        y,
        sr=sr,
        meter_map=meter_map,
        hop_length=HOP_LENGTH,
        bpm_threshold=BPM_THRESHOLD,
        chroma_type=CHROMA_TYPE,
        aggregate=AGGREGATE,
        accent_mode=ACCENT_MODE,
        weight_source=WEIGHT_SOURCE,
        weight_power=WEIGHT_POWER,
        min_frames_per_bin=MIN_FRAMES_PER_BIN,
    )

    chroma_results[track_name] = {
        "C_metric": C_metric,
        "audio_path": audio_path,
        "meter_path": meter_path,
    }

print(f"Computed chromagram for {len(chroma_results)} track(s)")

Found 12 track(s) with matching meter maps
  YTB-001: audio=YTB-001.wav meter=YTB-001_meter.npy
  YTB-002: audio=YTB-002.wav meter=YTB-002_meter.npy
  YTB-003: audio=YTB-003.wav meter=YTB-003_meter.npy
  YTB-004: audio=YTB-004.wav meter=YTB-004_meter.npy
  YTB-005: audio=YTB-005.wav meter=YTB-005_meter.npy
  YTB-006: audio=YTB-006.wav meter=YTB-006_meter.npy
  YTB-007: audio=YTB-007.wav meter=YTB-007_meter.npy
  YTB-008: audio=YTB-008.wav meter=YTB-008_meter.npy
  YTB-009: audio=YTB-009.wav meter=YTB-009_meter.npy
  YTB-011: audio=YTB-011.wav meter=YTB-011_meter.npy
  YTB-012: audio=YTB-012.wav meter=YTB-012_meter.npy
  YTB-013: audio=YTB-013.wav meter=YTB-013_meter.npy


ValueError: Subdivision bin too short: bin=2450, frames=1, min_frames_per_bin=2. (Consider smaller hop_length.)

In [ ]:
if chroma_results:
    n_tracks = len(chroma_results)
    fig, axes = plt.subplots(n_tracks, 1, figsize=(12, 2.5 * n_tracks), squeeze=False)

    chroma_labels = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

    for i, (track_name, data) in enumerate(chroma_results.items()):
        ax = axes[i, 0]
        C_metric = data["C_metric"]
        ax.imshow(C_metric, origin="lower", aspect="auto", cmap="magma")
        ax.set_title(f"{track_name} | shape={C_metric.shape}")
        ax.set_ylabel("Chroma")
        ax.set_yticks(np.arange(12))
        ax.set_yticklabels(chroma_labels)
        ax.set_xlabel("Metric bins")

    plt.tight_layout()
else:
    print("No chromagrams to plot.")

In [ ]:
CHROMA_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

for track_name, data in chroma_results.items():
    C_metric = data["C_metric"]
    out_path = CHROMA_OUTPUT_DIR / (
        f"{track_name}_chromagram_metric_{CHROMA_TYPE}_"
        f"{HOP_LENGTH}-{BPM_THRESHOLD}-{AGGREGATE}-{ACCENT_MODE}-{WEIGHT_SOURCE}-{WEIGHT_POWER}-{MIN_FRAMES_PER_BIN}.npy"
    )
    np.save(out_path, C_metric, allow_pickle=False)
    print(f"Saved: {out_path.name} shape={C_metric.shape}")